In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [8]:
import time
import asyncio
import datetime as dt
import gc
import numpy as np
import plotly.express as px
import pandas as pd
import ipywidgets as widgets

from helper_functions import mermaid
from model_classes import Scenario, multiple_replications
from distribution_classes import Exponential

Let's start with just having some patients arriving into our treatment centre.

In [4]:
mermaid("""
%%{ init: {  'flowchart': { 'curve': 'step'} } }%%

            %%{ init: {  'theme': 'base', 'themeVariables': {'lineColor': '#b4b4b4'} } }%%
            flowchart LR
            A[Arrival] --> B{Trauma or non-trauma}
            B --> B1{Trauma Pathway} 
            B --> B2{Non-Trauma Pathway}
                    
            B1 --> C[Stabilisation]
            C --> E[Treatment]
            E ----> F
                
            B2 --> D[Registration]
            D --> G[Examination]

            G --> H[Treat?]
            H ----> F[Discharge]
            H --> I[Non-Trauma Treatment]
            I --> F 

            C -.-> Z([Trauma Room])
            Z -.-> C

            E -.-> Y([Cubicle - 1])
            Y -.-> E

            D -.-> X([Clerks])
            X -.-> D

            G -.-> W([Exam Room])
            W -.-> G

            I -.-> V([Cubicle - 2])
            V -.-> I

            classDef highlight fill:#02CD55,stroke:#E8AD02,stroke-width:4px,color:#0C0D11,font-size:12pt,font-family:lexend;
            classDef unlight fill:#b4b4b4,stroke:#787878,stroke-width:2px,color:#787878,font-size:6pt,font-family:lexend;

            class A highlight;
            class B,B1,B2,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z unlight;
"""
)

To start with, we need to create some simulated patients who will turn up to our centre. 

To simulate patient arrivals, we will use the exponential distribution, which looks a bit like this. 

In [5]:
exp_dist = Exponential(mean=5)
exp_fig_example = px.histogram(exp_dist.sample(size=5000), 
                            width=600, height=300)

exp_fig_example.layout.update(showlegend=False, 
                    margin=dict(l=0, r=0, t=0, b=0))
exp_fig_example

To start with, we're just going to assume people arrive at a consistent rate throughout all 24 hours of the day. This isn't very realistic, but we can refine this later. 


When a patient arrives, the computer will pick a random number from this distribution to decide how long it will be before the next patient arrives at our treatment centre. 

Where the bar is very high, there is a high chance that the random number picked will be somewhere around that value. 

Where the bar is very low, it's very unlikely that the number picked will be from around that area - but it's not impossible. 

So what this ends up meaning is that, in this case, it's quite likely that the gap between each patient turning up at our centre will be somewhere between 0 and 10 minutes - and in fact, most of the time, someone will turn up every 2 or 3 minutes. However, now and again, we'll get a quiet period - and it might be 20 or 30 minutes until the next person arrives. 

This is quite realistic for a lot of systems - people tend to arrive fairly regularly, but sometimes the gap will be longer. 

As we get into more complex models, we can vary the distribution for different times of day or different months of the year so we can reflect real-world patterns better, but for now, we're just going to assume the arrival pattern is consistent. 

# Variability and Computers

Without getting too philosophical, the version of reality that happens is just one possible version!

Maybe we're going to get a really hot summer that means our department is busier due to heatstroke and people having accidents outside. 
Maybe it will rain all summer and everyone will stay indoors. 

And what if lots of people turn up really close together? How well does our department cope with that? 


So instead of just generating one set of arrivals, we will run the simulation multiple times. 

The first time the picks might be like this:

**5 minute gap, 4 minute gap, 5 minute gap, 6 minute gap**

The next time they might be like this:

**4 minute gap, 25 minute gap, 2 minute gap, 1 minute gap**

And so on. 

## Random seeds

Because computers aren't very good at being truly random, we give them a little nudge by telling them a 'random seed' to start from. 

You don't need to worry about how that works - but if our random seed is 1, we will draw a different set of times from our distribution to if our random seed is 100. 

This allows us to make lots of different realities! 

# Things to Try Out

- Try changing the slider with the title *'How many patients should arrive per day on average?'*. 
        
Look at the graph below it. The horizontal axis (the bottom one) shows the number of minutes  

How does the shape of the graph change when you change the value?

---
- Change the slider *'How many patients should arrive per day on average?'* back to the default (150) and click on 'Run simulation'. 
    - Look at the charts that show the variation in patient arrivals per simulation run. 
    - Look at the scatter (dot) plots at the bottom of the page to understand how the arrival times of patients varies across different simulation runs and different days. 
        - Hover over the dots to see more detail about the arrival time of each patient. By 6am, roughly how many patients have arrived in each simulation run? 
    - Think about how this randomness in arrival times across different runs could be useful.

---
- Try changing the random number the computer uses without changing anything else. What happens to the number of patients? Do the bar charts and histograms look different?


## Bonus Exercises

---        
- Try running the simulation for under 5 days. What happens to the height of the bars in the first bar chart compared to running the simulation for more days? Are the bars larger or smaller? 

---
- Try increasing the number of simulation runs. What do you notice about the *shape* of the histograms? Where are the bars highest?

In [10]:
seed = widgets.IntSlider(description="🎲 Set a random number for the computer to start from",
                        min=1, max=1000,
                        step=1, value=103)

run_time_days = widgets.IntSlider(description="🗓️ How many days should we run the simulation for each time?",
                            min=1, max=31,
                            step=1, value=15)

n_reps = widgets.IntSlider(description="🔁 How many times should the simulation run?",
                    min=1, max=25,
                    step=1, value=10)

mean_arrivals_per_day = widgets.IntSlider(description="🧍 How many patients should arrive per day on average?",
                                          min=60, max=300,
                                          step=5, value=80)

In [12]:
display(seed)
display(run_time_days)
display(n_reps)
display(mean_arrivals_per_day)

IntSlider(value=103, description='🎲 Set a random number for the computer to start from', max=1000, min=1)

IntSlider(value=15, description='🗓️ How many days should we run the simulation for each time?', max=31, min=1)

IntSlider(value=10, description='🔁 How many times should the simulation run?', max=25, min=1)

IntSlider(value=80, description='🧍 How many patients should arrive per day on average?', max=300, min=60, step…

In [13]:
exp_dist = Exponential(mean=60/(mean_arrivals_per_day.value/24), random_seed=seed.value)
exp_fig = px.histogram(exp_dist.sample(size=2500), 
                        width=500, height=250,
                        labels={
                "value": "Time between patients arriving (Minutes)"
            })

exp_fig.update_layout(yaxis_title="")

exp_fig.layout.update(showlegend=False, 
                        margin=dict(l=0, r=0, t=0, b=0))
exp_fig.update_xaxes(tick0=0, dtick=10, range=[0, 260])

exp_fig

In [15]:
args = Scenario(random_number_set=seed.value, 
                    # We want to pass the interarrival time here
                    # To get from daily arrivals to average interarrival time,
                    # divide the number of arrivals by 24 to get arrivals per hour,
                    # then divide 60 by this value to get the number of minutes
                    manual_arrival_rate=60/(mean_arrivals_per_day.value/24),
                    override_arrival_rate=True)

In [18]:
detailed_outputs = multiple_replications(
                args,
                n_reps=n_reps.value,
                rc_period=run_time_days.value*60*24,
                return_detailed_logs=True

            )

patient_log = pd.concat([detailed_outputs[i]['results']['full_event_log'].assign(Rep= i+1) 
                for i in range(n_reps.value)])

results = pd.concat([detailed_outputs[i]['results']['summary_df'].assign(rep= i+1)
                                    for i in range(n_reps.value)]).set_index('rep')


patient_log = patient_log.assign(model_day = (patient_log.time/24/60).pipe(np.floor)+1)
patient_log = patient_log.assign(time_in_day= (patient_log.time - ((patient_log.model_day -1) * 24 * 60)).pipe(np.floor))
# patient_log = patient_log.assign(time_in_day_ (patient_log.time_in_day/60).pipe(np.floor))
patient_log['patient_full_id'] = patient_log['Rep'].astype(str) + '_' + patient_log['patient'].astype(str) 
patient_log['rank'] = patient_log['time_in_day'].rank(method='max')


# Results

## Difference between average daily patients generated across simulation runs

The graph below shows the variation in the number of patients generated per day (on average) in a single simulation run. 
This can help us understand how much variation we get between model runs when we don't change parameters, only the random seed.

The height of each bar is relative to the **first** simulation run. 

A bar that is **positive** shows that **more** patients were generated on average per day in that simulation than in the first simulation.

A bar that is **negative** shows that **fewer* patients were generated on average per day in that simulation than in the first simulation.

In [20]:
results['00a_arrivals_difference'] = ((results[['00_arrivals']].iloc[0]['00_arrivals'].astype(int))/run_time_days.value) - (results['00_arrivals']/run_time_days.value) 

results["colour_00a"] = np.where(results['00a_arrivals_difference']<0, 'neg', 'pos')
# st.write(results)

run_diff_bar_fig = px.bar(results.reset_index(drop=False), 
                            x="rep", y='00a_arrivals_difference',
                            color="colour_00a")

run_diff_bar_fig.update_layout(
    yaxis_title="Difference in daily patients between first run and this run",
    xaxis_title="Simulation Run")


run_diff_bar_fig.layout.update(showlegend=False)
run_diff_bar_fig.update_xaxes(tick0=1, dtick=1)


run_diff_bar_fig

## Histogram: Total Patients per Run

This plot shows the variation in the **total** daily patients per run. 
                
The horizontal axis shows the range of patients generated within a simulation run.

The height of the bar shows how many simulation runs had an output in that group.

In [22]:
total_fig = px.histogram(
            results[['00_arrivals']],
            nbins=5
            )
total_fig.layout.update(showlegend=False)

total_fig.update_layout(yaxis_title="Number of Simulation Runs",
                        xaxis_title="Total Patients Generated in Run")

total_fig

## Histogram: Average Daily Patients per Run

This plot shows the variation in the **average** daily patients per run. 
                
The horizontal axis shows the range of patients generated within a simulation run

The height of the bar shows how many simulation runs had an output in that group.

In [24]:
daily_average_fig = px.histogram(
                    (results[['00_arrivals']]/run_time_days.value).round(1),
                        nbins=5
                    )
daily_average_fig.layout.update(showlegend=False)

daily_average_fig.update_layout(yaxis_title="Number of Simulation Runs",
                        xaxis_title="Average Daily Patients Generated in Run")

daily_average_fig

In [27]:
patient_log['minute'] = dt.date.today() + pd.DateOffset(days=165) +  pd.TimedeltaIndex(patient_log['time'], unit='m')
# https://strftime.org/
patient_log['minute_display'] = patient_log['minute'].apply(lambda x: dt.datetime.strftime(x, '%d %B %Y\n%H:%M'))
patient_log['minute_in_day'] = patient_log['minute'].apply(lambda x: dt.datetime.strftime(x, '%H:%M'))

The plots below show the minute-by-minute arrivals of patients across different model replications and different days.
Only the first 10 replications and the first 5 days of the model are shown. 

Each dot is a single patient arriving. 

From left to right within each plot, we start at one minute past midnight and move through the day until midnight. 

Looking from the top to the bottom of each plot, we have the model replications. 

Each horizontal line of dots represents a **full day** for one model replication.  

Hovering over the dots will show the exact time of each patient.

In [32]:
for i in range(5):
    minimal_log = patient_log[(patient_log['event'] == 'arrival') & 
                            (patient_log['Rep'] <= 10) & 
                            (patient_log['model_day'] == i+1)].copy()
    
    minimal_log['Rep_str'] = minimal_log['Rep'].astype(str)

    time_plot = px.scatter(
            minimal_log.sort_values("minute"),
            x="minute", 
            y="Rep",
            color="Rep_str",
            custom_data=["Rep", "minute_in_day", "patient"],
            category_orders={'Rep_str': [str(i+1) for i in range(10)]},
            range_y=[0.5, min(10, n_reps.value)+0.5],
            width=1200,
            height=300,
            opacity=0.5
    )
    
    del minimal_log

    time_plot.update_traces(
        hovertemplate="<br>".join([
            "Replication:%{customdata[0]}", 
            "Time of patient arrival: %{customdata[1]}",
            "Arrival in this simulation run: %{customdata[2]}"
        ])
    )

    time_plot.update_layout(yaxis_title="Simulation Run (Replication)",
                            xaxis_title="Time",
                            title="### Day {}".format(i+1),
                yaxis = dict(
                tickmode = 'linear',
                tick0 = 1,
                dtick = 1
            ))
    
    time_plot.layout.update(showlegend=False, 
                            margin=dict(l=0, r=0, t=0, b=0))
    
    time_plot.show()